<h1> Build bigram model for cell phone reviews </h1>

In [2]:
import psycopg2
import numpy as np
import json
from gensim import corpora, models, similarities
from gensim.models.doc2vec import LabeledSentence
import re

<h1> connect to postgres db and pull quesiton and answer text</h1>

In [3]:
conn = psycopg2.connect("host=localhost port=5432 dbname=amazon user=postgres password=darkmatter")
cur = conn.cursor()

In [4]:
cur.execute("SELECT * from metadata limit 1;")

In [5]:
cur.fetchall()

[(1784326,
  {'asin': '8282000477',
   'categories': [['Cell Phones & Accessories',
     'Accessories',
     'Chargers',
     'Car Chargers']],
   'imUrl': 'http://ecx.images-amazon.com/images/I/31B1s0wXl-L.jpg',
   'price': 13.79,
   'related': {'also_bought': ['B008EGWUJQ',
     'B008EGWURI',
     'B008NAYNBG',
     'B001CZBZOM',
     '8288836925',
     'B0091IYM0G'],
    'bought_together': ['B008EGWUJQ', 'B008NAYNBG'],
    'buy_after_viewing': ['B0091IYM0G',
     'B008EGWUJQ',
     'B008EGWURI',
     'B008NAYNBG']},
   'salesRank': {'Cell Phones & Accessories': 253289},
   'title': 'Samsung Official OEM Car Charger for your Jitterbug Phone! Original Equipment and Manufacturer (DC 12 volt)'},
  '8282000477',
  'Samsung Official OEM Car Charger for your Jitterbug Phone! Original Equipment and Manufacturer (DC 12 volt)')]

<h3> search on reviews_cell_phones_and_accessories </h3>

In [4]:
cur.execute("SELECT reviewText from reviews_cell_phones_and_accessories;")
#cur.execute("SELECT reviewText from reviews_electronics;")
RT_results=cur.fetchall()

#cur.execute("SELECT summary from reviews_cell_phones_and_accessories;")
#Sum_results=cur.fetchall()

<h1> clean up each question/answer and tokenize </h1>

In [5]:
def process_line(sentence):
    #step 1 split if we need to
    sentences=re.split(r"(?<![0-9])[.?!;](?![0-9])", sentence)
    result= [re.findall("[a-z'.-0-9]+", sent.lower()) for sent in sentences if \
            re.findall("[a-z'.-0-9]+", sent.lower())!=[]]
    if result==[]:
        result=[['']]
    return result
stoplist = set('for a of the and to in rt'.split())

<h1> Apply process line. Each sentence is treated as a seperate bag of words </h1>

In [6]:
#in this case all of the sentences in the reviews are treated as a unique sentence
review_sentences=[[[word for word in sentence if word not in stoplist] for sentence \
                       in process_line(review[0])] for review in RT_results]
list_of_senteces=[]
for review_line in review_sentences:
    for sentence in review_line:
        list_of_senteces.append(sentence)
############################################################
#else do this #in this case, all of the words in each sentence of a review are grouped together into one bag of words
#review_sentence= [[word for word in sum(process_line(sentence[0]),[]) if word not in complete_bag] for sentence in RT_results]

<h1> train word2Vec model </h1>

In [18]:
Rev_model = models.Word2Vec(review_sentences, size=100, window=5, min_count=10, workers=8)

In [20]:
Rev_model.most_similar('phone')

NameError: name 'Rev_model' is not defined

<h1> train biagram </h1>

In [17]:
Rbigram = models.Phrases(list_of_senteces,min_count=5) #model bigrams
Rbigrams=list(Rbigram[list_of_senteces]) #make list
RmodelB = models.Word2Vec(Rbigrams, size=100, window=5, min_count=5, workers=8) #train

In [21]:
RmodelB.most_similar('phone') #positive=['finna', 'gonna'], negative=['bro'],topn=20)

[('cellphone', 0.6505069732666016),
 ("phone's", 0.6486824154853821),
 ('device', 0.6273693442344666),
 ('iphone', 0.6251250505447388),
 ('evo', 0.6217646598815918),
 ('lap', 0.5964158177375793),
 ('it', 0.5708815455436707),
 ('shoulder', 0.5705478191375732),
 ('belt', 0.5587344169616699),
 ('wrist', 0.5546225309371948)]

In [19]:
RmodelB.save('/home/ubuntu/TallLabs/models/RmodelB_cell')